# Opening a Supermarket in Colombo


### Download and install the necessary libraries

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 

!conda install -c conda-forge geocoder --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### Scrapping data from Wikipedia

In [10]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Colombo").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []

for row in soup.find_all("div", class_="mw-content-ltr")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
cmb_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cmb_df.shape

(67, 1)

### Creating a function to extract longitudes and latitudes

In [11]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Colombo 7, Sri Lanka'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in cmb_df["Neighborhood"].tolist() ]

In [13]:
coords

[[6.883980000000065, 79.90080000000006],
 [6.8716700000000515, 79.99732000000006],
 [6.904710000000023, 79.85507000000007],
 [6.905200000000036, 79.91554000000008],
 [6.856881411183795, 79.89399987308153],
 [6.954920000000072, 79.86670000000004],
 [6.840800000000058, 79.90441000000004],
 [6.916690000000074, 79.87762000000004],
 [6.909510000000068, 79.86775000000006],
 [6.931940000000054, 79.84555000000006],
 [6.9705600000000345, 79.91224000000005],
 [6.851320000000044, 79.86590000000007],
 [6.840470000000039, 79.87825000000004],
 [6.937220000000025, 79.88221000000004],
 [6.94148000000007, 79.84664000000004],
 [6.946550000000059, 79.87033000000008],
 [6.887580000000071, 79.86256000000003],
 [6.880198044958258, 79.9473712755298],
 [6.936210000000074, 79.85845000000006],
 [6.931940000000054, 79.84555000000006],
 [6.856899611811464, 79.8785200945985],
 [6.936941510623433, 79.98104386711984],
 [6.7816400000000385, 79.98749000000004],
 [6.866550000000075, 79.87646000000007],
 [6.842750000000

### Concat latitude and longitude columns to the dataframe

In [14]:
df_coords=pd.DataFrame(coords,columns=['Latitude','Longitude'])
cmb_df['Latitude']=df_coords['Latitude']
cmb_df['Longitude']=df_coords['Longitude']
cmb_df.head()

,Neighborhood,Latitude,Longitude
0,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.883980,79.90080
1,Athurugiriya,6.871670,79.99732
2,Bambalapitiya,6.904710,79.85507
3,Battaramulla,6.905200,79.91554
4,Batuwatta,6.856881,79.89400


In [15]:
# get the coordinates of Colombo City 
address = 'Colombo City, Sri Lanka'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Colombo City, Sri Lanka {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Colombo City, Sri Lanka 6.9349969, 79.8538463.


In [16]:
# create map of Colombo using latitude and longitude values
map_cmb = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(cmb_df['Latitude'], cmb_df['Longitude'], cmb_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_cmb)  
    
map_cmb

In [17]:
map_cmb.save('map1.html')

### Connecting to foursquare

In [18]:
CLIENT_ID = 'XRXCIJLOUQSASIDPCK0UWRHNZRRR3F41AYXQKRJ0ADJ4MFB0' # your Foursquare ID
CLIENT_SECRET = 'YPML3OYG3JK25RPEHJNN214UM23UF35SSPCP5S2DGSBJYVFJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XRXCIJLOUQSASIDPCK0UWRHNZRRR3F41AYXQKRJ0ADJ4MFB0
CLIENT_SECRET:YPML3OYG3JK25RPEHJNN214UM23UF35SSPCP5S2DGSBJYVFJ


### Extract top 100 venues within 2000 meters

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(cmb_df['Latitude'], cmb_df['Longitude'], cmb_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4167, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.88398,79.9008,Nelum Koley Bath,6.878776,79.899975,Restaurant
1,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.88398,79.9008,Keels Super,6.883593,79.909762,Convenience Store
2,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.88398,79.9008,Sovereign German Restaurant,6.881046,79.890082,German Restaurant
3,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.88398,79.9008,Power World Nugegoda,6.874780,79.896036,Gym
4,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",6.88398,79.9008,Perera and Sons,6.883163,79.901816,Bakery


### Number of venues for each neighborhood

In [29]:
venues_df.groupby(["Neighborhood"]).count().reset_index()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Athurugiriya,17,17,17,17,17,17
1,Bambalapitiya,100,100,100,100,100,100
2,Battaramulla,78,78,78,78,78,78
3,Batuwatta,58,58,58,58,58,58
4,Bloemendhal,29,29,29,29,29,29
5,Boralesgamuwa,28,28,28,28,28,28
6,Borella,100,100,100,100,100,100
7,Cinnamon Gardens,100,100,100,100,100,100
8,Colombo,100,100,100,100,100,100
9,Dalugama,24,24,24,24,24,24


In [30]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 191 uniques categories.


In [31]:
# one hot encoding
cmb_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cmb_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cmb_onehot.columns[-1]] + list(cmb_onehot.columns[:-1])
cmb_onehot = cmb_onehot[fixed_columns]

print(cmb_onehot.shape)
cmb_onehot.head(35)

(4167, 192)


,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beach Bar,Bed & Breakfast,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dive Spot,Donut Shop,Electronics Store,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,National Park,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Pool,Pub,Racetrack,Record Shop,Recording Studio,Recreation Center,Resort,Rest Area,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Women's Store,Zoo
0,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"► Sri Jayawardenepura Kotte‎ (6 C, 4 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group rows by neighborhood and frequency of occurence of venues

In [32]:
cmb_grouped = cmb_onehot.groupby(["Neighborhood"]).mean().reset_index()
cmb_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beach Bar,Bed & Breakfast,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Camera Store,Candy Store,Casino,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dive Spot,Donut Shop,Electronics Store,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,National Park,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pier,Pizza Place,Platform,Playground,Pool,Pub,Racetrack,Record Shop,Recording Studio,Recreation Center,Resort,Rest Area,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Women's Store,Zoo
0,Athurugiriya,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.058824,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.058824,0.058824,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.058824,0.235294,0.000000,

In [33]:
kd_grouped = cmb_grouped[["Supermarket"]]

In [34]:
len(kd_grouped[kd_grouped["Supermarket"] > 0])

60

### Create a dataset with Neighborhood and Supermarket

In [35]:
kd_market = cmb_grouped[["Neighborhood","Supermarket"]]
kd_market.head()

,Neighborhood,Supermarket
0,Athurugiriya,0.235294
1,Bambalapitiya,0.000000
2,Battaramulla,0.038462
3,Batuwatta,0.120690
4,Bloemendhal,0.034483


### Use K-mean algorithm to cluster the neighborhoods w.r.t Supermarket abundancy

In [36]:
# set number of clusters
kclusters = 3

kd_clustering = kd_market.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 1, 0, 1, 2, 2, 2, 2])

In [37]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kd_merged = kd_market.copy()

# add clustering labels
kd_merged["Cluster Labels"] = kmeans.labels_
kd_mod=kd_merged[['Supermarket','Cluster Labels']]

# We can group supermarket abundancy w.r.t clusters
kd_mod.groupby(['Cluster Labels']).mean()

,Supermarket
Cluster Labels,
0,0.051112
1,0.129264
2,0.011652


### Merging the cluster column to the dataframe

In [38]:
kd_merged = kd_merged.join(cmb_df.set_index("Neighborhood"), on="Neighborhood")

print(kd_merged.shape)
kd_merged.head()

(67, 5)


,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,Athurugiriya,0.235294,1,6.871670,79.99732
1,Bambalapitiya,0.000000,2,6.904710,79.85507
2,Battaramulla,0.038462,0,6.905200,79.91554
3,Batuwatta,0.120690,1,6.856881,79.89400
4,Bloemendhal,0.034483,0,6.954920,79.86670


In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kd_merged['Latitude'], kd_merged['Longitude'], kd_merged['Neighborhood'], kd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
map_clusters.save('clustered_map.html')

### Cluster 0

In [41]:
kd_merged.loc[kd_merged['Cluster Labels'] == 0]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
2,Battaramulla,0.038462,0,6.905200,79.915540
4,Bloemendhal,0.034483,0,6.954920,79.866700
12,Dematagoda,0.040000,0,6.937220,79.882210
19,Kadawatha,0.040000,0,6.856900,79.878520
22,Kalubowila,0.040000,0,6.866550,79.876460
25,Kiribathgoda,0.076923,0,6.982370,79.927540
26,Kirulapana,0.040000,0,6.878604,79.873269
27,Kohuwala,0.034884,0,6.869590,79.888900
29,Kolonnawa,0.047619,0,6.932620,79.888860
30,Koswatte,0.042105,0,6.901910,79.894130


### Cluster 1

In [42]:
kd_merged.loc[kd_merged['Cluster Labels'] == 1]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,Athurugiriya,0.235294,1,6.871670,79.997320
3,Batuwatta,0.120690,1,6.856881,79.894000
5,Boralesgamuwa,0.107143,1,6.840800,79.904410
16,Hokandara,0.150000,1,6.880198,79.947371
20,"Kaduwela, Western Province",0.100000,1,6.936942,79.981044
32,Kotikawatta,0.100000,1,6.931380,79.916180
33,Kottawa,0.105263,1,6.841360,79.965200
36,Maharagama,0.114286,1,6.846740,79.927110
42,Moratuwa,0.115385,1,6.778920,79.883090
49,Pannipitiya,0.153846,1,6.845370,79.939010


### Cluster 2

In [43]:
kd_merged.loc[kd_merged['Cluster Labels'] == 2]

,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
1,Bambalapitiya,0.000000,2,6.904710,79.855070
6,Borella,0.000000,2,6.916690,79.877620
7,Cinnamon Gardens,0.020000,2,6.909510,79.867750
8,Colombo,0.010000,2,6.931940,79.845550
9,Dalugama,0.000000,2,6.970560,79.912240
10,Dehiwala,0.022727,2,6.851320,79.865900
11,Dehiwala-Mount Lavinia,0.030000,2,6.840470,79.878250
13,Fort (Colombo),0.014286,2,6.941480,79.846640
14,Grandpass,0.026316,2,6.946550,79.870330
15,Havelock Town,0.010000,2,6.887580,79.862560


# Conclusion

The neighborhoods belonging to the cluster 2 has the minium number of or no supermarkets. Therefore the option available for project developers is to build supermarkets in one of the neighborhoods that belongs to cluster 2.